In [5]:
#%% Imports
import pycromanager
import numpy as np
import imagej
import os
import time

In [31]:
#%% Autofocus_WS_plane

from pycromanager import Core

mmc = Core()
for _ in dir(mmc):
    if not _.startswith('_'):
        print(_)

add_galvo_polygon_vertex
clear_circular_buffer
clear_roi
clone
create_tagged_image
debug_log_enabled
define_config
define_config_group
define_pixel_size_config
define_state_label
delete
delete_config
delete_config_group
delete_galvo_polygons
delete_pixel_size_config
detect_device
device_busy
device_type_busy
display_slm_image
enable_continuous_focus
enable_debug_log
enable_feature
enable_stderr_log
equals
finalize
full_focus
get_allowed_property_values
get_api_version_info
get_auto_focus_device
get_auto_focus_offset
get_auto_shutter
get_available_config_groups
get_available_configs
get_available_device_descriptions
get_available_device_types
get_available_devices
get_available_pixel_size_configs
get_buffer_free_capacity
get_buffer_total_capacity
get_bytes_per_pixel
get_c_ptr
get_camera_channel_name
get_camera_device
get_channel_group
get_circular_buffer_memory_footprint
get_class
get_config_data
get_config_group_state
get_config_group_state_from_cache
get_config_state
get_core_error_te

In [8]:
#%% Autofocus_WS_plane

import matplotlib.pyplot as plt
import tifffile as tiff
import numpy as np
from os.path import join
from pycromanager import Core
from skimage import io, color

# directory for saving and reading images
directory = "C:/Users/19719431\OneDrive - Maynooth University/Desktop/Automated_Raman"
def brightfield_autofocus(path) -> None:
    core = Core()
    # core.load_system_configuration("core.config_demo.cfg")

    # Record images along z axis
    # core.set_property("TransmittedLamp", "Label", "On")
    # core.set_camera_device("AmScope")
    # camera_name = core.get_camera_device()
    # core.set_property(camera_name, "Binning", "1x1")
    # core.set_property(camera_name, "PixelType", "GREY8")
    # core.set_property(camera_name, "ExposureAuto", "0")
    # core.set_serial_port_command("COM13", "CURRENT=0", "\r\n")
    width = core.get_image_width()
    height = core.get_image_height()
    byte_depth = core.get_bytes_per_pixel()

    xystage = core.get_xy_stage_device()
    zstage = core.get_focus_device()
    pos = {'z': core.get_position(zstage)}
    nz = 60  # number of images
    step = 5
    pos['z'] -= nz * 0.5 * step

    for a in range(nz + 2):
        core.snap_image()
        # core.snap_image()
        # time.sleep(0.05)
        img = core.get_image()
        
        if byte_depth == 1:
            ip = np.reshape(img, (height, width)).astype(np.uint8)
        elif byte_depth == 2:
            ip = np.reshape(img, (height, width)).astype(np.uint16)
        else:
            raise ValueError(f'byte depth should be 1 or 2. byte depth: {byte_depth}')

        if a == 0 or a == 1:
            pre_path = join(path, f"Autofocus/image_{a}.tif")
        else:
            pre_path = join(path, f"Autofocus/image_{a-2}.tif")

        tiff.imwrite(pre_path, ip, photometric='minisblack')
        pos['z'] += step
        # core.set_position(zstage, pos['z'])
        # time.sleep(0.05)

    pos['z'] -= nz * step
    # core.set_position(zstage, pos['z'])

    # Find minimum variance corresponds to in-focus plane
    # core.set_property("TransmittedLamp", "Label", "Off")
    # core.set_camera_device("Andor")
    core.snap_image()

    image = tiff.imread(join(path, "Autofocus/image_0.tif"))
    mean, std = np.mean(image), np.std(image)
    norm_var = std * std / mean

    min_var, min_var_index = norm_var, 0
    for i in range(1, nz):
        image = tiff.imread(join(path, f"Autofocus/image_{i}.tif"))
        mean, std = np.mean(image), np.std(image)
        norm_var = std * std / mean
        if norm_var < min_var:
            min_var, min_var_index = norm_var, i

    pos['z'] += step * (min_var_index) - 10
    # core.set_position(zstage, pos['z'])
    # time.sleep(0.5)


brightfield_autofocus(directory)

In [ ]:
#%% image_grid -> cell_filter_isolation

# Initialize PyImageJ !! problem exist here
ij = imagej.init('sc.fiji:fiji')

# Initialize Micro-Manager
with pycromanager.Server() as server:
    mmc = server.get_core()
    gui = server.get_studio().app()

    # Camera and stage settings
    width = mmc.getImageWidth()
    height = mmc.getImageHeight()
    byteDepth = mmc.getBytesPerPixel()
    cameraName = mmc.getCameraDevice()
    mmc.setProperty(cameraName, "Binning", "1x1")
    mmc.setProperty(cameraName, "PixelType", "GREY8")
    mmc.setProperty(cameraName, "ExposureAuto", "0")
    mmc.setExposure(25)
    xystage = mmc.getXYStageDevice()

    pos_x = mmc.getXPosition(xystage)
    pos_y = mmc.getYPosition(xystage)

    Xorigin = pos_x
    Yorigin = pos_y

    # Grid settings
    n = 5  # Number of columns
    m = 4  # Number of rows
    incX = 320  # Step size in X direction
    incY = 240  # Step size in Y direction
    directory = "/path/to/save/directory/"

    pos_x = pos_x - incX * (0.5 * n - 0.5)
    startx = pos_x
    pos_y = pos_y - incY * (0.5 * m - 0.5)
    starty = pos_y

    cc = 0

    gui.setXYStagePosition(pos_x, pos_y)
    time.sleep(1)

    for a in range(m):
        for b in range(n):
            pos_x = startx + incX * b
            pos_y = starty + incY * a

            gui.setXYStagePosition(pos_x, pos_y)
            time.sleep(0.05)
            mmc.snapImage()
            mmc.snapImage()
            time.sleep(0.05)
            img = mmc.getImage()

            if byteDepth == 1:
                ip = ij.py.new_image(width, height, 1)
                ip.getProcessor().setPixels(np.array(img).reshape(height, width))
            elif byteDepth == 2:
                ip = ij.py.new_image(width, height, 2)
                ip.getProcessor().setPixels(np.array(img).reshape(height, width))
            else:
                print("Unknown byte depth.")
                exit()

            prePath = os.path.join(directory, f"Stitches/imageXY-{cc}.tif")
            imp = ij.py.imageplus(prePath, ip)
            ij.py.run_plugin('ij.io.FileSaver', imp, [prePath])
            cc += 1

    gui.setXYStagePosition(Xorigin, Yorigin)

In [ ]:
#%% stitch_images -> cell_filter_isolation
from ij.measure import ResultsTable
from ij.plugin.frame import RoiManager
from ij import IJ

# Function to perform stitching and find cell positions
def stitch_and_find_cells(directory, n, m):
    input_dir = os.path.join(directory, "Stitches")
    generic_filename = "imageXY-{i}.tif"
    stitch_image = os.path.join(input_dir, "stitchedImage.tif")

    # Run ImageJ stitching plugin
    ij.py.run_plugin("Clear Results", "")
    stitch_command = (
        f"type=[Grid: row-by-row] order=[Right & Up] grid_size_x={n} grid_size_y={m} "
        f"tile_overlap=90 first_file_index_i=1 directory=[{input_dir}] file_names={generic_filename} "
        f"output_textfile_name=_ambient_stitched.txt fusion_method=[Linear Blending] regression_threshold=0.30 "
        f"max/avg_displacement_threshold=2.50 absolute_displacement_threshold=3.50 compute_overlap "
        f"subpixel_accuracy computation_parameters=[Save computation time (but use more RAM)] "
        f"image_output=[Fuse and display]"
    )
    IJ.run("Grid/Collection stitching", stitch_command)
    IJ.saveAs("Tiff", stitch_image)

    # Run point selection on stitched image
    stitched_image = IJ.openImage(stitch_image)
    stitched_image.show()
    IJ.run(stitched_image, "Find Maxima...", "noise=35 output=[Point Selection]")
    IJ.run(stitched_image, "Measure")

    # Get results from the ResultsTable
    rt = ResultsTable.getResultsTable()
    num = rt.getCounter()

    # Extract coordinates from ResultsTable
    coordinates = []
    for k in range(num):
        x = int(rt.getValue("X", k))
        y = int(rt.getValue("Y", k))
        coordinates.append((x, y))

    # Clean up
    stitched_image.close()
    IJ.run("Clear Results")

    return coordinates

# Example usage
directory = "/path/to/save/directory/"
n = 5  # Number of columns
m = 4  # Number of rows

coordinates = stitch_and_find_cells(directory, n, m)
print("Detected cell positions:", coordinates)


In [1]:
#%% cell_filter_isolation

# Sample cell coordinates
cellXY = [
    [1041, 2009], [86, 1984], [1412, 1982], [1003, 1979], [2030, 1971],
    [1273, 1960], [1477, 1954], [2356, 1918], [2070, 1949], [2080, 1947],
    [2028, 1937], [1522, 1937], [2305, 1929], [946, 1912], [213, 1894],
    [505, 1853], [2182, 1830], [582, 1821], [530, 1811], [2192, 1802],
    [538, 1790], [170, 1787], [2090, 1784], [2220, 1780], [214, 1775]
]

n_filtered = 19

n_cells = len(cellXY)
if n_cells < n_filtered:
    n_filtered = n_cells

cellPos = np.array(cellXY)
minDistance = np.full(n_cells, np.inf)

# Find the nearest neighbor distance for each cell
for i in range(n_cells):
    for j in range(n_cells):
        if i != j:
            distance = np.linalg.norm(cellPos[i] - cellPos[j])
            if distance < minDistance[i]:
                minDistance[i] = distance

# Filter out the most isolated cells
minDistanceSorted = np.sort(minDistance)
threshold = minDistanceSorted[-n_filtered]

cellFiltered = []
for i in range(n_cells):
    if minDistance[i] >= threshold and len(cellFiltered) < n_filtered:
        cellFiltered.append(cellPos[i])

# Convert filtered cells to a list of tuples for easier handling
cellFiltered = [tuple(cell) for cell in cellFiltered]

# Return list of X most isolated cells
print("Most isolated cells:", cellFiltered)


In [ ]:
#%% cell_probe_order_NN

# Sample cell coordinates
n_cells = len(cellPos)
isVisited = np.zeros(n_cells, dtype=bool)  # Track visited cells
NN_list_abs = np.zeros((n_cells, 2), dtype=int)  # NN list

# Set the first cell in the NN list
NN_list_abs[0] = cellPos[0]
isVisited[0] = True
testCellX, testCellY = cellPos[0]

# Nearest Neighbor algorithm
for i in range(1, n_cells):
    minDistance = float('inf')
    minIndex = -1
    for j in range(n_cells):
        if not isVisited[j]:
            distance = np.linalg.norm([testCellX - cellPos[j][0], testCellY - cellPos[j][1]])
            if distance < minDistance:
                minDistance = distance
                minIndex = j
    
    NN_list_abs[i] = cellPos[minIndex]
    isVisited[minIndex] = True
    testCellX, testCellY = cellPos[minIndex]

# Save results to file
directory = "/path/to/save/directory"
results_path = os.path.join(directory, "Results", "cell_Positions.txt")
os.makedirs(os.path.dirname(results_path), exist_ok=True)

with open(results_path, 'w') as f:
    for pos in NN_list_abs:
        f.write(f"{pos[0]},{pos[1]}\n")

print(f"NN list saved to {results_path}")


In [ ]:
#%% laser_offset

with pycromanager.Core() as mmc:
    # Set up properties and devices
    mmc.setProperty("TransmittedLamp", "Label", "Off")
    mmc.setSerialPortCommand("COM13", "CURRENT=40", "\r\n")
    mmc.setCameraDevice("AmScope")
    exposure = mmc.getExposure()
    mmc.setExposure(25)
    time.sleep(5)

    # Get image dimensions
    width = int(mmc.getImageWidth())
    height = int(mmc.getImageHeight())

    # Snap image
    mmc.snapImage()
    image1 = mmc.getImage()
    time.sleep(0.5)
    mmc.snapImage()
    image2 = mmc.getImage()

    # Convert the image to an ImageJ object
    ip = ij.py.to_imagej(image2)
    ij.ui().show("Laser Image", ip)

    # Run Find Maxima and Measure
    ij.IJ.run(ip, "Find Maxima...", "noise=500 output=[Point Selection]")
    rt = ij.measure.ResultsTable.getResultsTable()
    num = rt.size()

    # Find the center of the laser spot
    laserSpot = [int(rt.getValue("X", 0)), int(rt.getValue("Y", 0))]

    # Calculate the offset between the laser spot and the center of the image
    laserOffset = [laserSpot[0] - width // 2, laserSpot[1] - height // 2]

    # Close the results and reset properties
    ij.IJ.run("Close")
    ij.IJ.run("Clear Results")
    mmc.setProperty("TransmittedLamp", "Label", "On")
    mmc.setExposure(exposure)

print(f"Laser offset from image center: {laserOffset}")


In [ ]:
#%% tilt_plane_function -> coordinate_scaling

mmc = pycromanager.Core()
gui = pycromanager.Acquisition()

n_cells = len(callPos)
edgeCell = np.zeros((3, 3))

# Find the two cells that are furthest apart
maxDistance = 0
for i in range(n_cells):
    for j in range(n_cells):
        distance = np.linalg.norm(cellStage[i] - cellStage[j])
        if distance > maxDistance:
            maxDistance = distance
            edgeCell[0][:2] = cellStage[i]
            edgeCell[1][:2] = cellStage[j]

# Find the cell whose sum of distance from the other two cells is greatest
maxSumDistance = 0
for i in range(n_cells):
    sumDistance = np.linalg.norm(cellStage[i] - edgeCell[0][:2]) + np.linalg.norm(cellStage[i] - edgeCell[1][:2])
    if sumDistance > maxSumDistance:
        maxSumDistance = sumDistance
        edgeCell[2][:2] = cellStage[i]

# Move to each cell and perform autofocus routine
# Assuming zstage is defined and autofocus_image_focus_plane.bsh sets the focus position
for i in range(3):
    gui.setXYStagePosition(edgeCell[i][0], edgeCell[i][1])
    # Run autofocus routine (implement this as needed)
    # exec(open(directory + "autofocus_image_focus_plane.py").read())
    # Simulating autofocus by setting z-position manually
    edgeCell[i][2] = mmc.getPosition('Z')  # replace 'Z' with actual z-stage device name

# Use X, Y, and in-focus Z of each cell to create a tilt plane
A = (edgeCell[1][1] - edgeCell[0][1]) * (edgeCell[2][2] - edgeCell[0][2]) - (edgeCell[1][2] - edgeCell[0][2]) * (edgeCell[2][1] - edgeCell[0][1])
B = (edgeCell[1][0] - edgeCell[0][0]) * (edgeCell[2][2] - edgeCell[0][2]) - (edgeCell[1][2] - edgeCell[0][2]) * (edgeCell[2][0] - edgeCell[0][0])
C = (edgeCell[1][0] - edgeCell[0][0]) * (edgeCell[2][1] - edgeCell[0][1]) - (edgeCell[1][1] - edgeCell[0][1]) * (edgeCell[2][0] - edgeCell[0][0])
D = edgeCell[0][1] * B - edgeCell[0][0] * A - edgeCell[0][2] * C

# The plane is defined by the equation Ax + By + Cz + D = 0
# To find the difference in Z between two points:
# delZ = -A(delX)/C - B(delY)/C

def tilt_plane_function(x1, y1, x2, y2):
    delX = x2 - x1
    delY = y2 - y1
    delZ = -A * delX / C - B * delY / C
    return delZ

# Example usage
x1, y1 = 100, 200  # replace with actual coordinates
x2, y2 = 300, 400  # replace with actual coordinates
relative_focus = tilt_plane_function(x1, y1, x2, y2)
print(f"Relative focus difference: {relative_focus:.2f} µm")

# Return tilt plane function or any other necessary result
tilt_plane_params = {'A': A, 'B': B, 'C': C, 'D': D}
print(f"Tilt plane parameters: {tilt_plane_params}")


In [ ]:
#%% coordinate_scaling

gui = pycromanager.Studio()

directory = "your_directory_path"  # Set your directory path here
xystage = mmc.getXYStageDevice()
n = 2  # Set grid size n
m = 2  # Set grid size m
incX = 400  # Set increment value
objRatio = 1.0  # Object ratio, adjust as needed
objShift = [0.0, 0.0]  # Set object shift if needed
n_cells = len(NN_list_abs)

# Read pixel shift data from the image stitching routine
coords = []
with open(f"{directory}Stitches/_ambient_stitched.registered.txt", 'r') as file:
    for line in file.readlines():
        if '(' in line:
            coord = line[line.index('(')+1:line.index(')')].split(',')
            coords.append([float(coord[0]), float(coord[1])])

coords = np.array(coords)

# Get central coordinates from both image and stage
imp = ij.io().open(f"{directory}Stitches/stitchedImage.tif")
stats = ij.op().run("statistics.regular", imp.getProcessor())
ctrImage = [0.5 * stats[4], 0.5 * stats[5]]  # stats[4] and stats[5] are image width and height

ctrStage = [mmc.getXPosition(xystage), mmc.getYPosition(xystage)]

# Calculate rotation and scaling vector
theta = math.atan2(coords[n-1, 1], coords[n-1, 0])
SinO = math.sin(theta)
CosO = math.cos(theta)
scaler = (n - 1) * incX / np.linalg.norm(coords[n-1])

offset = [-(scaler * ctrImage[0] * CosO + scaler * ctrImage[1] * SinO),
          -(-scaler * ctrImage[0] * SinO + scaler * ctrImage[1] * CosO)]

laserOffset = [0, 0]  # Assuming laserOffset is defined somewhere
laserOffsetStage = [objRatio * (scaler * laserOffset[0] * CosO + scaler * laserOffset[1] * SinO),
                    objRatio * (-scaler * laserOffset[0] * SinO + scaler * laserOffset[1] * CosO)]

objShiftStage = [scaler * objShift[0] * CosO + scaler * objShift[1] * SinO,
                 -scaler * objShift[0] * SinO + scaler * objShift[1] * CosO]

# Apply transform matrix to cell coordinates
cellStage = np.zeros((n_cells, 2))
for i in range(n_cells):
    cellStage[i, 0] = scaler * NN_list_abs[i][0] * CosO + scaler * NN_list_abs[i][1] * SinO + offset[0] - laserOffsetStage[0] + ctrStage[0] + objShiftStage[0]
    cellStage[i, 1] = -(-scaler * NN_list_abs[i][0] * SinO + scaler * NN_list_abs[i][1] * CosO) - offset[1] + laserOffsetStage[1] + ctrStage[1] - objShiftStage[1]

# Write the transformed cell locations to a file
with open(f"{directory}\\Results\\cell_locations.txt", 'w') as file:
    for k in range(n_cells):
        file.write(f"{cellStage[k, 0]},{cellStage[k, 1]}\n")


In [ ]:
#%% record_spectra -> save to disk

gui = pycromanager.events.gui()
directory = "your_directory_path"  # Set your directory path here
camera_name = mmc.get_camera_device()
width = mmc.get_image_width()
height = mmc.get_image_height()
byte_depth = mmc.get_bytes_per_pixel()
mmc.set_property(camera_name, "ReadMode", "FVB")
mmc.set_exposure(10000)
xystage = mmc.get_xy_stage_device()
zstage = mmc.get_focus_device()
pos = {'x': mmc.get_x_position(xystage), 'y': mmc.get_y_position(xystage), 'z': mmc.get_position(zstage)}

# Declare variables
n_cells = len(cellStage)
cellStageRel = []
spectra = []

# Move to cell position and record spectrum
mmc.set_xy_position(cellStage[0][0], cellStage[0][1])
mmc.wait_for_device(zstage)
mmc.execute("file save")
time.sleep(10)  # Adjust sleep time if needed
WSPlane = mmc.get_position(zstage)
mmc.execute("file save")
time.sleep(10)  # Adjust sleep time if needed
IFPlane = mmc.get_position(zstage)

d = 0  # debug counter
mmc.set_camera_device("Andor")
for b in range(n_cells):
    print(d)
    mmc.set_property("TransmittedLamp", "Label", "Off")
    mmc.set_serial_port_command("COM13", "CURRENT=100", "\r\n")
    time.sleep(2)

    mmc.snap_image()
    image = mmc.get_image()
    spectra.append(image)

    if b < n_cells - 1:
        mmc.set_xy_position(cellStage[b][0], cellStage[b][1])
        time.sleep(0.05)

mmc.set_serial_port_command("COM13", "CURRENT=0", "\r\n")
mmc.set_camera_device("AmScope")

with open(f"{directory}/Results/{time.strftime('%Y%m%d_%H%M%S')}_spectra.txt", "a") as f:
    for k in range(n_cells):
        for j in range(width):
            f.write(f"{spectra[k][j]},")
        f.write("\n")

mmc.set_xy_position(pos['x'], pos['y'])
